# EventPipe Throughput Testing

The cells below can be pointed at a process and will bucket event counts into "seconds since start of trace".  This will output a graph of Events Per Second.

In [ ]:
#r "nuget:Microsoft.Diagnostics.NETCore.Client,0.2.61701"
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent,2.0.49"


using Microsoft.Diagnostics.NETCore.Client;
using System.Diagnostics;
using Microsoft.Diagnostics.Tracing;
using Microsoft.Diagnostics.Tracing.Parsers;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using System.Threading.Tasks;
using System.Collections.Generic;
using System.Linq;
using XPlot.Plotly;

Change the value of `procName` to the name of your process.

Alternatively, use a different identifier, e.g., PID, to find your target.

In [ ]:
// Put your process name here
var procName = "ConsoleApp";

var availableProcs = DiagnosticsClient.GetPublishedProcesses()
    .Select(x => Process.GetProcessById(x))
    .Select(p => new { name = p.ProcessName, id = p.Id, module = p.MainModule.FileName });

foreach (var p in availableProcs)
    display(p);

The cell below will create an EventPipe session looking for a provider named `MySource`.  Change the provider list for testing different combinations of providers.

The callback for `EventPipeEventSource` buckets all events to the second in the trace in which they happened.

In [ ]:
var client = new DiagnosticsClient(availableProcs.Where(p => p.name == procName).FirstOrDefault()?.id ?? -1);
var session = client.StartEventPipeSession(
    providers: new List<EventPipeProvider> 
    { 
        // new EventPipeProvider("Microsoft-Windows-DotNETRuntime", System.Diagnostics.Tracing.EventLevel.Verbose, (long)(-1)),
        // new EventPipeProvider("Microsoft-DotNETCore-SampleProfiler", System.Diagnostics.Tracing.EventLevel.Verbose, (long)(-1)),
        new EventPipeProvider("MySource", System.Diagnostics.Tracing.EventLevel.Verbose, (long)(-1))
    }
);

var eventDict = new Dictionary<double, int>();

var source = new EventPipeEventSource(session.EventStream);
Action<TraceEvent> processEvent = (TraceEvent obj) =>
{
    var t = Math.Truncate(obj.TimeStamp.TimeOfDay.TotalSeconds);
    
    if (eventDict.TryGetValue(t, out var count))
    {
        eventDict[t] += 1;
    }
    else
    {
        eventDict[t] = 1;
    }
};

source.Dynamic.All += processEvent;

var readTask = Task.Run(() => 
{
    try
    {
        source.Process();
    }
    catch (Exception e)
    {
        display(e);
    }
});
    

Task.Delay(TimeSpan.FromMinutes(7)).Wait();
// source.StopProcessing();
session.Stop();
display("Stopping");
readTask.Wait();
display("Stopped");

In [ ]:
// use this cell for debugging your data
foreach (var (key,val) in eventDict)
{
    display($"{key}, {val}");
}

In [ ]:
// Plot the Events/Second data.
Chart.Plot(
    new Graph.Scatter()
    {
        y = eventDict.Values
    }
)

In [ ]:
// Use this cell for exporting your data to a string for easy re-use

using System.Text.Json;
using System.Text.Json.Serialization;

var timeSeriesData = eventDict.Select((kv, _) => kv.Value);

string str = JsonSerializer.Serialize(timeSeriesData);
display(str);